In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, img_to_array,ImageDataGenerator
from evaluate import evaluate
from utils import img_parts_generator, top3acc, top5acc, labels_contiguous
from inceptionv4 import create_model, two_towers_top
from scipy.misc import imsave
import time
import os
from keras import backend as K
import pickle

Using TensorFlow backend.


In [2]:
def plot_data(img, orig_x1, orig_y1, orig_x2, orig_y2, pred_x1, pred_y1, pred_x2, pred_y2):
    # Create figure and axes
    fig,ax = plt.subplots(1)

    # Display the image
    ax.imshow(img)
    orig_width = orig_x2 - orig_x1
    orig_height = orig_y2 - orig_y1
    # Create a Rectangle patch
    rect = patches.Rectangle((orig_x1, orig_y1),orig_width, orig_height,linewidth=2,edgecolor='g',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

    pred_width = pred_x2 - pred_x1
    pred_height = pred_y2 - pred_y1
    rect = patches.Rectangle((pred_x1, pred_y1),pred_width,pred_height,linewidth=2,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()
    
def plot_img(img, x=None, y=None, save=None):
    implot = plt.imshow(img)
    if x is not None and y is not None:
        plt.plot(x,y,'o', marker=5)
    if save:
        imsave(img,path)
    plt.show()

In [3]:
# load model
weights = 'best_weights/complete_defrost_62.hdf5'
model = create_model(num_classes=500,
                     include_top=True, 
                     weights=weights)
print(model.summary())

Inceptionv4 Base loaded


/home/ubuntu/birdsnap_dataset/deepbirdsnap/inceptionv4.py:400: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  top_model = Model(input=inputs, output=x)
/home/ubuntu/birdsnap_dataset/deepbirdsnap/inceptionv4.py:465: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`
  fused = Model(input=base_inputs, output=top(base(base_inputs)))


Weights loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
inception_v4 (Model)         (None, 8, 8, 1536)        41174400  
_________________________________________________________________
model_1 (Model)              (None, 500)               768500    
Total params: 41,942,900.0
Trainable params: 41,879,732.0
Non-trainable params: 63,168.0
_________________________________________________________________
None


In [4]:
# load data
gen_factory = ImageDataGenerator(rescale=1./255)

validation_generator = gen_factory.flow_from_directory(
    '../validation',
    target_size=(299, 299),
    batch_size=100,
    shuffle=False)

test_generator = gen_factory.flow_from_directory(
    '../test',
    target_size=(299, 299),
    batch_size=100,
    shuffle=False)



Found 3000 images belonging to 500 classes.
Found 4500 images belonging to 500 classes.


In [4]:
valid_labels = labels_contiguous(6,num_classes=500)
test_labels = labels_contiguous(9,num_classes=500)

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy', top3acc, top5acc])

In [6]:
# evaluate model on val and test sets
res_val = model.evaluate_generator(validation_generator, 3000//100)
print(res_val)
res_test = model.evaluate_generator(test_generator, 4500//100)
print(res_test)

[1.7215816140174867, 0.61666663388411203, 0.79699997107187903, 0.84633332490921021]
[1.8312264177534314, 0.60733331044514971, 0.78066664801703556, 0.83666665951410935]


In [7]:
preds = model.predict_generator(validation_generator, 3000//100, verbose=1)

30/30 [==============================] - 264s   


In [20]:
len(valid_labels[0])

500

In [41]:
def get_label_name(index):
    f = open('train_labels.txt', 'r').readlines()
    s = set()
    for c in f:
        s.add(c)
    mapping = list(s)
    mapping.sort()
    return mapping[index]

In [34]:
#load data from custom generator
gen = img_parts_generator('parts_info.txt', 
                          data_dir='../validation/', 
                          batch_size=100, load_image=True, load_paths=False, load_parts=False, cache=False)

In [35]:
preds_gen = None
count = 0
for imgs in gen:
    #print(type(imgs[0]))
    prediction = model.predict(imgs[0], 100)
    #print(np.argmax(prediction).shape)
    if preds_gen is None:
        preds_gen = prediction
    else:
        preds_gen = np.vstack((preds_gen, prediction))
    #print(type(preds_gen))
    count += 100
    print(count)
    


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [36]:
mistakes = 0
index = []
for i, (pred,tgt) in enumerate(zip(preds_gen, valid_labels)):
    #print(i)
    if np.argmax(pred) != np.argmax(tgt):
        mistakes += 1
        index.append(i)
print(mistakes)
print(len(index))


1150
1150


In [48]:
print(get_label_name(np.argmax(valid_labels[1780])))

Northern_Bobwhite



In [49]:
print(get_label_name(np.argmax(preds_gen[1780])))

Sooty_Grouse



In [45]:
#load data from custom generator
index_set = set(index)
gen = img_parts_generator('parts_info.txt', 
                          data_dir='../validation/', 
                          batch_size=1, load_image=True, load_paths=True, load_parts=False, cache=False)

In [46]:
import re

In [47]:
count = 0
wrong_classified_path = []
for imgs, paths in gen:
    img = imgs[0]
    path = paths[0]
    if count in index_set:
        if '333760' in path:
            print("Found: ", count)
        wrong_classified_path.append(path)
    count += 1
    if count%50 == 0:
        print(count)
len(wrong_classified_path)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
Found:  1780
1800


KeyboardInterrupt: 

In [38]:
wrong_classified_path[0]

'../validation/Acadian_Flycatcher/534084.jpg'

In [42]:
with open('wrong_classified_60','wb') as f:
    pickle.dump(wrong_classified_path,f)

In [3]:
wrong_classified_path = pickle.load(open('wrong_classified_60', 'rb'))

In [4]:
from shutil import copyfile, copy2
from image_cropper import get_save_path
save_path = '../wrong_classified_60'

In [1]:
for path in wrong_classified_path:
    new_path = get_save_path(path, save_path)
    copyfile(path, new_path)

    

In [50]:
validation_data = np.load(open('/data/bottlenecks_bak/cropped_pred_77_validation.npy', 'rb'))
validation_data2 = np.load(open('/data/bottlenecks_bak/bottleneck_60_validation.npy', 'rb'))

In [51]:
validation_labels = labels_contiguous(6,num_classes=500)
validation_labels2 = labels_contiguous(6,num_classes=500)

In [52]:
top_model, x, top_model_inputs = two_towers_top(weights='best_weights/two_towers_top_predAndWhole82.hdf5')

/home/ubuntu/birdsnap_dataset/deepbirdsnap/inceptionv4.py:336: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  top_model = Model(input=[input1,input2], output=x)


Loaded top model weights


In [12]:
validation_data.shape

(3000, 8, 8, 1536)

In [53]:
top_model.compile(optimizer='adam',
  loss='categorical_crossentropy')

In [54]:
gen = img_parts_generator('parts_info.txt', 
                          data_dir='../validation/', 
                          batch_size=10, load_image=True, load_paths=False, load_parts=False, cache=False)

In [55]:
preds_gen = None
count = 0
for imgs in gen:
    
    v1 = validation_data[count:count+10, :, :, :]
    v2 = validation_data2[count:count+10, :, :, :]
    v = [v1, v2]
    prediction = top_model.predict(v, 10)
    
    if preds_gen is None:
        preds_gen = prediction
    else:
        preds_gen = np.vstack((preds_gen, prediction))
    #print(type(preds_gen))
    count += 10
    print(count)
    


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [56]:
mistakes = 0
index = []
for i, (pred,tgt) in enumerate(zip(preds_gen, validation_labels)):
    #print(i)
    if np.argmax(pred) != np.argmax(tgt):
        mistakes += 1
        index.append(i)
print(mistakes)
print(len(index))


549
549


In [67]:
print(get_label_name(np.argmax(preds_gen[1877])))
print(get_label_name(np.argmax(validation_labels[1877])))

Bewicks_Wren

Olive_Sparrow



In [57]:
import re

In [66]:
index_set_twin = set(index)
gen = img_parts_generator('parts_info.txt', 
                          data_dir='../validation/', 
                          batch_size=1, load_image=True, load_paths=True, load_parts=False, cache=False)

count = 0
wrong_classified_path = []
for imgs, paths in gen:
    img = imgs[0]
    path = paths[0]
    if count in index_set_twin:
        if "576390" in path:
            print("Found: ", count)
        wrong_classified_path.append(path)
    count += 1
    if count%50 == 0:
        print(count)
len(wrong_classified_path)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
Found:  1877
1900
1950
2000
2050
2100
2150


KeyboardInterrupt: 

In [19]:
from shutil import copyfile, copy2
from image_cropper import get_save_path
save_path = '../top_two_towers_82'

In [20]:
for path in wrong_classified_path:
    new_path = get_save_path(path, save_path)
    copyfile(path, new_path)

    

../top_two_towers_82/validation/Acadian_Flycatcher
../top_two_towers_82/validation/Acadian_Flycatcher
../top_two_towers_82/validation/Acadian_Flycatcher
../top_two_towers_82/validation/Acadian_Flycatcher
../top_two_towers_82/validation/Acadian_Flycatcher
../top_two_towers_82/validation/Alder_Flycatcher
../top_two_towers_82/validation/Alder_Flycatcher
../top_two_towers_82/validation/Alder_Flycatcher
../top_two_towers_82/validation/Alder_Flycatcher
../top_two_towers_82/validation/Allens_Hummingbird
../top_two_towers_82/validation/Allens_Hummingbird
../top_two_towers_82/validation/Allens_Hummingbird
../top_two_towers_82/validation/Allens_Hummingbird
../top_two_towers_82/validation/Altamira_Oriole
../top_two_towers_82/validation/American_Black_Duck
../top_two_towers_82/validation/American_Crow
../top_two_towers_82/validation/American_Crow
../top_two_towers_82/validation/American_Golden_Plover
../top_two_towers_82/validation/American_Golden_Plover
../top_two_towers_82/validation/American_Go